In [3]:
from tkinter import *
from tkinter import colorchooser
from PIL import ImageTk, Image
%run ./base_exercises.ipynb
%run ./exercise_data.ipynb

# Global variable to keep track of the current exercise index
# When moving to the next/prev exercise, this index changes accordingly
current_index = 0

# Dictionary to store text colors for each exercise because each might have different colors
text_colors = {}

# Dictionary to store current colors for different parts of the exercise details
current_colors = {
    'title_label': 'black',
    'name_label': 'black',
    'name': 'black',
    'description_label': 'black',
    'description': 'black',
    'duration_label': 'black',
    'duration': 'black',
    'difficulty_label': 'black',
    'difficulty': 'black',
    'muscles_label': 'black',
    'muscles': 'black',
    'instructions_label': 'black',
    'instructions_text': 'black',
    'instructions': 'black',
    'textcolor': 'red'
}

# Global variable for title text
title_text = "Fitness for All!"

# Create the main window
window = Tk()
window.title("Exercise Details")

# Get the screen width & height
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

# Setting the window size to fill the entire screen
window.geometry(f"{screen_width}x{screen_height}")

icon = PhotoImage(file='../Shared Components/Assets/workout_icon.png')
window.iconphoto(True, icon)

# Load the background image
image_path = "../Shared Components/Assets/gym_s2.png"
bg_image = PhotoImage(file=image_path)
bg_label = Label(window, image=bg_image, name="bg_label")
bg_label.place(relwidth=1, relheight=1)
bg_label.image = bg_image  # Keep a reference to avoid garbage collection


# Function to display exercise details in the window
def display_exercise_details():
    global current_index, title_text

    exercise = exercises[current_index]

    # Initialize color dictionary for current exercise if not already done
    if exercise.name not in text_colors:
        text_colors[exercise.name] = current_colors.copy()  # Creates a new dict with default colors for the current exercise

    # Clear previous labels and texts
    for widget in window.winfo_children():
        if str(widget) != ".bg_label":
            widget.destroy()

    # Function to choose color
    def choose_color(label_key, label):
        color_code = colorchooser.askcolor(title="Choose Your Favorite Color!")[1]
        if color_code:
            label.config(fg=color_code)
            text_colors[exercise.name][label_key] = color_code
            # Update current colors to reflect the new choice for subsequent exercises
            current_colors[label_key] = color_code

    # Create labels to display exercise details
    def create_label(parent, text, row, column, font=("Verdana", 15, 'bold'), label_key=None, colspan=1, width=None, height=None, sticky=None):
        # Determine the text color to use for this label
        fg_color = text_colors[exercise.name].get(label_key, current_colors[label_key])

        # Create the label with the specified text, font, and color
        label = Label(parent, text=text, font=font, bg='#45A3FD', fg=fg_color, cursor="hand2",
                      borderwidth=10, relief=FLAT, padx=10, pady=10, width=width, height=height)

        # Place the label in the grid layout at the specified row/column
        label.grid(row=row, column=column, columnspan=colspan, sticky=sticky, padx=5, pady=5)

        # If label_key provided, make the label clickable to change its color
        if label_key:
            label.bind("<Button-1>", lambda e: choose_color(label_key, label))

        return label
    
    def set_bg_image(frame, image_path):
        bg_image = PhotoImage(file=image_path)
        bg_label = Label(frame, image=bg_image)
        bg_label.image = bg_image
        bg_label.place(relwidth=1, relheight=1)

    # Title Frame
    title_frame = Frame(window, borderwidth=10, relief=RAISED)
    title_frame.grid(row=0, column=0, columnspan=3, padx=5, pady=5)
    
    set_bg_image(title_frame, '../Shared Components/Assets/puzzle_bg.png')
    
    title_label = Label(title_frame, text=title_text, font=("Verdana", 15, 'bold'), fg=current_colors['title_label'], cursor='hand2')
    title_label.pack(padx=10, pady=10)
    title_label.bind("<Button-1>", lambda e: choose_color('title_label', title_label))

    # Encapsulating frame test
    new_frame = Frame(window, bg="red", borderwidth=10, relief=RIDGE)
    new_frame.place(x=50, y=150)

    
    # Name, Duration, and Difficulty Frames
    name_duration_difficulty_frame = Frame(new_frame, borderwidth=10, relief=RIDGE)
    name_duration_difficulty_frame.grid(row=0, column=0, sticky='nsew')
    
    set_bg_image(name_duration_difficulty_frame, '../Shared Components/Assets/puzzle_bg.png')

    # Name Frame
    name_frame = Frame(name_duration_difficulty_frame, bg='#24404B', borderwidth=10, relief=RIDGE)
    name_frame.grid(row=0, column=0, sticky='nsew')
    create_label(name_frame, "Name: " + exercise.name, 0, 0, label_key='name_label', sticky='w')

    # Duration Frame
    duration_frame = Frame(name_duration_difficulty_frame, bg='#24404B', borderwidth=10, relief=RIDGE)
    duration_frame.grid(row=0, column=1, sticky='nsew')
    create_label(duration_frame, "Duration: " + str(exercise.duration) + " seconds", 0, 0, label_key='duration_label', sticky='w')

    # Difficulty Frame
    difficulty_frame = Frame(name_duration_difficulty_frame, bg='#24404B', borderwidth=10, relief=RIDGE)
    difficulty_frame.grid(row=0, column=2, sticky='nsew')
    create_label(difficulty_frame, "Difficulty: " + exercise.difficulty, 0, 0, label_key='difficulty_label', sticky='w')

    # Description Frame
    description_frame = Frame(new_frame, borderwidth=10, relief=RIDGE)
    description_frame.grid(row=1, column=0, sticky='nsew')
    create_label(description_frame, "Description: " + exercise.description, 0, 0, label_key='description_label', sticky='w')
    
    # set_bg_image(description_frame, '../Shared Components/Assets/puzzle_bg.png')

    # Muscle Group + Click on Text frame
    muscle_text_frame = Frame(new_frame, bg='#45A3FD', borderwidth=10, relief=RIDGE)
    muscle_text_frame.grid(row=2, column=0, sticky='nsew' )
    
    set_bg_image(muscle_text_frame, '../Shared Components/Assets/puzzle_bg.png')

    # Muscle Groups Frame
    muscle_groups_frame = Frame(muscle_text_frame, bg='#24404B', borderwidth=10, relief=RIDGE)
    muscle_groups_frame.grid(row=2, column=0, sticky='nsew')
    muscle_groups_str = ", ".join(exercise.muscle_groups)
    create_label(muscle_groups_frame, "Muscles Worked: " + muscle_groups_str, 0, 0, label_key='muscles_label', sticky='w')

    # Configure grid row and column weights to make them expandable
    new_frame.grid_rowconfigure(0, weight=1)
    new_frame.grid_rowconfigure(1, weight=1)
    new_frame.grid_rowconfigure(2, weight=1)
    new_frame.grid_columnconfigure(0, weight=1)

    # Instructions Label Frame
    instructions_label_frame = Frame(window, bg='#24404B', borderwidth=10, relief=RIDGE)
    instructions_label_frame.grid(row=6, column=0, columnspan=2, sticky='w', padx=5, pady=5)

    # Instructions Label
    create_label(instructions_label_frame, "Instructions:", 6, 0, label_key='instructions_label', sticky='w')
        
    # Frame to hold the Text widget and the scrollbar
    instructions_frame = Frame(window)
    instructions_frame.grid(row=6, column=1, sticky="w")

    instructions_text = Text(instructions_frame, height=10, width=40, wrap=WORD,
                             bg='#027878',
                             fg=text_colors[exercise.name]['instructions'],
                             cursor="hand2",
                             borderwidth=10,
                             relief=SUNKEN)

    instructions_text.insert(END, exercise.instructions)
    instructions_text.config(state=DISABLED, font=("Verdana", 15, 'bold'))  # Make the text entry field read-only
    instructions_text.grid(row=0, column=0, sticky="w")
    instructions_text.bind("<Button-1>", lambda e: choose_color('instructions', instructions_text))

    scrollbar = Scrollbar(instructions_frame, command=instructions_text.yview)
    instructions_text.config(yscrollcommand=scrollbar.set)
    scrollbar.grid(row=0, column=1, sticky='ns')

    # Click on text Label
    click_text_frame = Frame(muscle_text_frame, bg='#24404B', borderwidth=10, relief=RIDGE)
    click_text_frame.grid(row=2, column=1, sticky='nsew')

    create_label(click_text_frame, "Click on any text to change its color :D", 0, 0, font=("Verdana", 13), label_key='textcolor', sticky='nsew')


    # Load/Displaying the Workout Images
    img = Image.open(exercise.image_path)
    img = img.resize((300, 300), Image.LANCZOS)
    photo_img = ImageTk.PhotoImage(img)
    img_label = Label(window, image=photo_img)
    img_label.image = photo_img
    img_label.place(x=1200, y=200)

    # Creating Navigation Buttons
    prev_btn = Button(window, text="Previous", command=show_prev_exercise, width=15, height=5,
                      cursor="hand2", relief=RAISED, borderwidth=10, font=("Verdana", 13))
    prev_btn.grid(row=7, column=0, sticky="w", padx=5, pady=30)

    next_btn = Button(window, text="Next", command=show_next_exercise, width=15, height=5,
                      cursor="hand2", relief=RAISED, borderwidth=10, font=("Verdana", 13), activebackground="yellow")
    next_btn.grid(row=7, column=5, sticky="e")

# Function to show previous exercise
def show_prev_exercise():
    global current_index, animate_flag
    animate_flag = False  # Stop the current animation
    current_index = (current_index - 1) % len(exercises)
    display_exercise_details()

# Function to show next exercise
def show_next_exercise():
    global current_index, animate_flag
    animate_flag = False  # Stop the current animation
    current_index = (current_index + 1) % len(exercises)
    display_exercise_details()

# Initial display of exercise details
display_exercise_details()

# Configure grid row and column weights to make them expandable
for row in range(8):
    window.grid_rowconfigure(row, weight=1)
for column in range(3):
    window.grid_columnconfigure(column, weight=1)

# Start the main loop
window.mainloop()


